In [4]:
# First cell: import necessary modules
%load_ext autoreload
%autoreload 2

import cv2
import numpy as np
import os
import sys
import matplotlib.pyplot as plt
import pickle
import time

from backbone import Backbone
from candidate_point import CandidatePoints
from candidate_point_detect import CandidatePointsDetect
from constant import WORM, ROOT_SMOOTH
from graph import Graph
from graph_builder import GraphBuilder
from graph_prune import GraphPrune
from root_smooth import RootSmooth
from search_backbone import SearchBackbone
from skimage import io, color, morphology, measure
from scipy.ndimage import distance_transform_edt
from skimage import color, measure, morphology
from scipy.ndimage import distance_transform_edt
from concurrent.futures import ProcessPoolExecutor
import multiprocessing
import pickle

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


ModuleNotFoundError: No module named 'backbone'

ERROR: Could not find a version that satisfies the requirement backbone (from versions: none)
ERROR: No matching distribution found for backbone


In [ ]:
# Second cell: utility functions
def load_skip_list(filename):
    skip_list = []
    try:
        with open(filename, 'r') as file:
            for line in file:
                skip_list.append(int(line.strip()))
    except IOError:
        pass
    return skip_list

def image_get(file_path, img_index):
    # First try the filename with zfill(5)
    image_filename_zfill = f"{file_path}/{str(img_index).zfill(5)}.png"
    image_filename = f"{file_path}/{img_index}.png"
    print(image_filename_zfill)
    if os.path.exists(image_filename_zfill):
        image_filename = image_filename_zfill
    elif not os.path.exists(image_filename):
        print(f"Pic {img_index} : Not Exist!")
        return None

    image = cv2.imread(image_filename, cv2.IMREAD_GRAYSCALE)

    if image is None:
        print(f"Pic {img_index} : Failed to Load!")
        return None

    gray = (image < 163) & (image > 161)
    image = measure.label(gray).astype(np.uint8)

    if image.shape[0] != WORM.IMAGE_SIZE2 or image.shape[1] != WORM.IMAGE_SIZE1:
        print(f"Pic {img_index} : Wrong Image Size!")
        return None

    return image


def image_get2(file_path, img_index):
    image_filename = f"{file_path}/{str(img_index).zfill(5)}.png"
    image = cv2.imread(image_filename, cv2.IMREAD_GRAYSCALE)
    gray =  (image < 179) & (image > 177)
    image = measure.label(gray).astype(np.uint8)

    if image is None or image.shape[0] != WORM.IMAGE_SIZE2 or image.shape[1] != WORM.IMAGE_SIZE1:
        print(f"Pic {img_index} : Not Exist or Wrong Image Size 1!")
        return None
    return image

def image_get1(file_path, img_index):
    image_filename = f"{file_path}/{str(img_index).zfill(5)}.png"
    image = cv2.imread(image_filename, cv2.IMREAD_GRAYSCALE)

    if image is None:
        print(f"Pic {img_index} : Not Exist!")
        return None

    # Assuming we want to take the left half of the image
    left_half_image = image[:, :image.shape[1] // 2]

    gray = (left_half_image < 163) & (left_half_image > 161)
    labeled_image = measure.label(gray).astype(np.uint8)


    return labeled_image

def in_skip_list(index, skip_list):
    return index in skip_list


# Save results to CSV
def save_to_csv(save_path, image_path, ordered_points, max_thickness, length):
    frame_number = os.path.basename(image_path).split('.')[0].replace('frame', '')
    filename = os.path.join(save_path, "results.csv")
    ordered_points_flattened = [coord for point in ordered_points for coord in point]

    with open(filename, 'a') as file:
        file.write(','.join([frame_number] + [str(max_thickness)] + [str(length)] + list(map(str, ordered_points_flattened))) + '\n')



# Example gradient function
def interpolate_color(start_color, end_color, factor):
    return tuple([start_color[i] + factor * (end_color[i] - start_color[i]) for i in range(3)])

# Normalize the color values to 0-1 range for matplotlib
def normalize_color(color):
    return [c / 255.0 for c in color]

In [ ]:
pic_start =0
pic_end = 9898

start_color = (255, 0, 0)  # Red
end_color = (0, 0, 255)  # Blue


PIC_START = pic_start
PIC_END = pic_end


# '/home/agross/Segment-and-Track-Anything/tracking_results/RIM+RIB/worm_worm-1519-0'
def process_folder(folder):
    search_backbone = SearchBackbone()
    A = folder[63:]+'_masks'
    print(A)
    file_path = os.path.join(folder, A)
    save_folder = os.path.join(folder, 'results')
    if not os.path.exists(save_folder):
        os.makedirs(save_folder)

    for pic_num in range(PIC_START, PIC_END + 1):
        image = image_get(file_path, pic_num)

        print()
        if image is not None:
            filename = f"{file_path}/backbone_{pic_num}.bin"

            try:
                start = time.time()
                backbone = search_backbone.search(image)
                end = time.time()
                print(f"Centerline extraction time consumption: {(end - start) * 1000}ms {pic_num}")

                distance_transform = distance_transform_edt(image)
                thicknesses = [distance_transform[int(y), int(x)] for y, x in search_backbone.temp_backbone.cood]
                max_thickness = max(thicknesses)
                length = search_backbone.temp_backbone.wormLength
                print(length)
                print(max_thickness)
                save_to_csv(save_folder, f"{file_path}/frame{pic_num}.png", search_backbone.temp_backbone.cood, max_thickness, length)

            except Exception as e:
                print(f"Error: {e}")
            search_backbone.save_centerline_results(filename)

# List of folders to process
folders = [
    '/home/agross/Segment-and-Track-Anything2/tracking_results/done/worm-0342-0',
    '/home/agross/Segment-and-Track-Anything2/tracking_results/done/worm-1338-0',
    '/home/agross/Segment-and-Track-Anything2/tracking_results/done/worm-2401-0',
    '/home/agross/Segment-and-Track-Anything2/tracking_results/done/worm-7918-0',
    '/home/agross/Segment-and-Track-Anything2/tracking_results/done/worm-8224-0',
    '/home/agross/Segment-and-Track-Anything2/tracking_results/done/worm-9240-0'
'/home/agross/Segment-and-Track-Anything2/tracking_results/worm-3212-0' '/home/agross/Segment-and-Track-Anything2/tracking_results/worm-4337-0' '/home/agross/Segment-and-Track-Anything2/tracking_results/worm-5636-0' '/home/agross/Segment-and-Track-Anything2/tracking_results/worm-6014-0'

]
for folder in folders:
    process_folder(folder)

worm-0342-0_masks
/home/agross/Segment-and-Track-Anything2/tracking_results/done/worm-0342-0/worm-0342-0_masks/00000.png

Error: 'NoneType' object has no attribute 'degree'
/home/agross/Segment-and-Track-Anything2/tracking_results/done/worm-0342-0/worm-0342-0_masks/00001.png

170
Current backbone length: 335.3555328415488, Mean length: 0.0
Centerline extraction time consumption: 238.66009712219238ms 1
335.3555328415488
15.231546211727817
/home/agross/Segment-and-Track-Anything2/tracking_results/done/worm-0342-0/worm-0342-0_masks/00002.png

101
158
11
158
158
Current backbone length: 331.00397963511125, Mean length: 335.3555328415488
Centerline extraction time consumption: 238.86823654174805ms 2
331.00397963511125
15.231546211727817
/home/agross/Segment-and-Track-Anything2/tracking_results/done/worm-0342-0/worm-0342-0_masks/00003.png

101
161
11
161
161
Current backbone length: 322.3325643561961, Mean length: 333.17975623833
Centerline extraction time consumption: 243.19100379943848ms 3

In [ ]:
folders = [
    '/home/agross/Segment-and-Track-Anything2/tracking_results/worm-3212-0',
    '/home/agross/Segment-and-Track-Anything2/tracking_results/worm-5636-0',
    '/home/agross/Segment-and-Track-Anything2/tracking_results/worm-6014-0',
    '/home/agross/Segment-and-Track-Anything2/tracking_results/worm-4337-0',

]
for folder in folders:
    process_folder(folder)